In [1]:
import papermill as pm

In [ ]:
path_to_default_config = 'config.yml'

In [ ]:
pm.execute_notebook(
   'path/to/input.ipynb',
   'path/to/output.ipynb',
   parameters = dict(alpha=0.6, ratio=0.1)
)